# Setup Steps

In [1]:
import docker
import sqlite3
client = docker.from_env()
conn = sqlite3.connect('dockermgr.db')

# Menu Steps

In [ ]:
conn.execute('''CREATE TABLE APPS
         (ID CHAR(10) PRIMARY KEY,
         APPNAME        TEXT    NOT NULL,
         DEFAULT_DUR    INT     NOT NULL,
         NBR_PORTS      INT    NOT NULL,
         MESSAGE        TEXT NOT NULL);''')

In [51]:
conn.execute("INSERT INTO APPS (ID,APPNAME,DEFAULT_DUR,STARTUP) \
      VALUES (null, 'MongoDB', 60, 'docker run -it -d -P --name mongodb mongodb:latest ')")
conn.commit()

In [52]:
app_cursor = conn.execute("SELECT id, appname, default_dur, startup from APPS")
for row in app_cursor:
   print ("ID = ", row[0])
   print ("Application = ", row[1])
   print ("Duration = ", row[2])
   print ("Command = ", row[3], "\n")

ID =  1
Application =  NGINX
Duration =  60
Command =  docker run -it -d --name nginx -P nginx:latest 

ID =  2
Application =  MongoDB
Duration =  60
Command =  docker run -it -d -P --name mongodb mongodb:latest  



# List Containers

In [64]:
import pprint
import random
pp = pprint.PrettyPrinter(indent=2)

cont_curr = client.containers.list()
for row in cont_curr:
#    print(row)
    print(str(row)[12:-1])
    container = client.containers.get(str(row)[12:-1])
#    pp.pprint(container.attrs)
    print('Name =',container.attrs['Name'][1:])
    print('Image =',container.attrs['Config']['Image'])
    ports = container.attrs['NetworkSettings']['Ports']
    print('Port =', ports[list(ports.keys())[0]][0]["HostPort"])
    print(ports[list(ports.keys())[0]][0]["HostPort"])
    


3457bb09d9
Name = MongoExpress-LAB01K9TB
Image = mongo-express
Port = 55002
55002
fcb2da1d7c
Name = Mongo-LAB01K9TB
Image = poc2opslabs/lab01:latest
Port = 55001
55001
aa4604cf7f
Name = here
Image = nginx
Port = 55000
55000
55002
55001
55000
N 59305


In [66]:
def check_port(a):
    checkflag = 'Y'
    cont_curr = client.containers.list()
    for row in cont_curr:
        container = client.containers.get(str(row)[12:-1])
        ports = container.attrs['NetworkSettings']['Ports']
        portstr = ports[list(ports.keys())[0]][0]["HostPort"]
        if portstr == str(yy):
            checkflag='N'  
    return checkflag

In [71]:
yy=random.randint(55000,55005)
print(check_port(yy), yy)  

Y 55004


# Managed Subscriptions

In [136]:
conn.execute('''CREATE TABLE SUBSCRIPTION
     (ID CHAR(20) PRIMARY KEY,
      APPNAME      TEXT NOT NULL,
      STARTTIME    DATETIME NOT NULL,
      DEFAULT_DUR  INT NOT NULL,
      MESSAGE.     TEXT NOT NULL);''')

In [174]:
conn.execute("INSERT INTO SUBSCRIPTION (ID,APPNAME,STARTTIME,DEFAULT_DUR) \
      VALUES ('2ede09df70b', 'NGINX', datetime('now'), 60)")
conn.commit()


In [184]:
app_cursor = conn.execute('''SELECT id, appname, starttime, default_dur from SUBSCRIPTION 
       where DATETIME('now') > DATETIME(starttime,"+"||default_dur||" minute");''')
for row in app_cursor:
    print ("ID =          ", row[0])
    print ("Application = ", row[1])
    print ("Start Time =  ", row[2])
    print ("Duration =    ", row[3], "\n")
    conn.execute('DELETE from SUBSCRIPTION WHERE ID = "'+row[0]+'"' )
conn.commit()

In [183]:
app_cursor = conn.execute('''SELECT id, appname, starttime, default_dur, 
       DATETIME(starttime,"+"||default_dur||" minute") as endtime, DATETIME('now') as nowtime from SUBSCRIPTION 
       where DATETIME('now') <= DATETIME(starttime,"+"||default_dur||" minute");''')
for row in app_cursor:
    print ("ID =          ", row[0])
    print ("Application = ", row[1])
    print ("Start Time =  ", row[2])
    print ("Duration =    ", row[3])
    print ("End Time =    ", row[4])
    print ("Now =         ", row[5], "\n")


ID =           2ede09df70b
Application =  NGINX
Start Time =   2021-02-27 01:05:00
Duration =     60
End Time =     2021-02-27 02:05:00
Now =          2021-02-27 01:08:18 



# Database Stuff

In [47]:
conn.execute('''CREATE TABLE APPS
         (ID INTEGER PRIMARY KEY AUTOINCREMENT,
         APPNAME        TEXT    NOT NULL,
         DEFAULT_DUR    INT     NOT NULL,
         STARTUP        CHAR(2000) NOT NULL);''')

In [43]:
conn.close()

# Other Stuff

In [9]:
client.images.pull('ubuntu')

<Image: 'ubuntu:latest'>

In [10]:
client.images.list()

[<Image: 'nginx:latest'>, <Image: 'ubuntu:latest'>]

In [11]:
client.containers.run("ubuntu:latest", "echo hello world")

b'hello world\n'

In [ ]:
events = client.events()
for event in events:
    print(event)
events.close()

In [32]:
import psutil
loadrate = float(list(psutil.getloadavg())[1])/float(psutil.cpu_count())
if loadrate >= 1:
    print('disabled')
else:
    print('enabled')

#x = list(psutil.getloadavg())[1] 
#print(float(x))

enabled


In [42]:
import shutil
print(str(round(list(shutil.disk_usage("/"))[2]/(1024*1024*1024),2))+' gb')

18.26Gb


In [46]:
len(client.containers.list())

1

In [47]:
len(client.images.list())

2

In [10]:
import base64

data = 1234567

# URL and Filename Safe Base64 Encoding
urlSafeEncodedBytes = base64.urlsafe_b64encode(str(data).encode("utf-8"))
urlSafeEncodedStr = str(urlSafeEncodedBytes, "utf-8")

print(urlSafeEncodedStr)

MTIzNDU2Nw==


In [25]:
import string    
import random  
ran = ''.join(random.choices(string.ascii_uppercase + string.digits + string.ascii_lowercase, k = 6))    
print("The randomly generated string is : " + str(ran)) # print the random data 

The randomly generated string is : 9VDHCv


In [23]:
len(string.ascii_uppercase + string.digits + string.ascii_lowercase)**6

56800235584

In [8]:
import subprocess

list_files = subprocess.run(["pwd"], stdout=subprocess.PIPE)
print("The exit code was: %d" % list_files.returncode)
print(list_files.stdout.decode('UTF-8'))

The exit code was: 0
/Users/rorymclean/notebooks



In [ ]:
import subprocess

lablaunch = subprocess.run(["./runlab.sh", 'LAB01', 'XYZZY', '55555', '55556', '', '', ''], stdout=subprocess.PIPE)
print("The exit code was: %d" % lablaunch.returncode)
print(list_lablaunchfiles.stdout.decode('UTF-8'))

In [ ]:
import subprocess

list_dir = subprocess.Popen(["sudo", "du", "-d", "1", "/Users/rorymclean/"])
print(list_dir.poll())
print(list_dir.wait())
print(list_dir.poll())
print(list_dir.stdout.decode('UTF-8'))

In [13]:
import onetimepad
import random
import string
 
def random_string_generator(str_size, allowed_chars):
    return ''.join(random.choice(allowed_chars) for x in range(str_size))

chars = string.ascii_letters #+ string.punctuation
clear_text = 'LAB01-456dyr'
pad_text = random_string_generator(len(clear_text), chars)
print(pad_text)
cipher = onetimepad.encrypt(clear_text, pad_text)
print(cipher)
msg = onetimepad.decrypt(cipher, pad_text)
print(msg)

MtDWsOaRlWvE
01350667426255675a330f37
LAB01-456dyr


In [5]:
import psutil
print(psutil.cpu_count())
print(psutil.cpu_count(logical=False))
print(psutil.cpu_times())
print(psutil.getloadavg())
print(psutil.cpu_percent(interval=1))
#print(psutil.cpu_percent(interval=0, percpu=True))

1
1
scputimes(user=4557.64, nice=27.11, system=762.79, idle=67004.67, iowait=2689.59, irq=0.0, softirq=55.75, steal=0.0, guest=0.0, guest_nice=0.0)
(0.49, 0.38, 0.21)
13.0


In [2]:
import psutil
print(psutil.users())
print(psutil.test())

[suser(name='rmclean', terminal=':0', host='localhost', started=1614890240.0, pid=2219)]
USER         PID  %MEM     VSZ     RSS  NICE STATUS  START   TIME  CMDLINE
root           1   0.1  100.0M   11.5M        sleep  Mar04  09:31  /sbin/init splash
root           2   0.0    0.0B    0.0B        sleep  Mar04  00:00  kthreadd
root           3   0.0    0.0B    0.0B   -20   idle  Mar04  00:00  rcu_gp
root           4   0.0    0.0B    0.0B   -20   idle  Mar04  00:00  rcu_par_gp
root           8   0.0    0.0B    0.0B   -20   idle  Mar04  00:00  mm_percpu_wq
root           9   0.0    0.0B    0.0B        sleep  Mar04  00:02  ksoftirqd/0
root          10   0.0    0.0B    0.0B         idle  Mar04  00:23  rcu_sched
root          11   0.0    0.0B    0.0B        sleep  Mar04  00:00  migration/0
root          12   0.0    0.0B    0.0B        sleep  Mar04  00:00  idle_inject/0
root          14   0.0    0.0B    0.0B        sleep  Mar04  00:00  cpuhp/0
root          15   0.0    0.0B    0.0B        sleep 

root         762   0.1  234.4M   10.1M        sleep  Mar04  00:04  /usr/sbin/ModemManager --filter-policy=strict
root         772   0.2  786.9M   48.8M        sleep  Mar04  05:48  /usr/bin/containerd
root         782   0.0  234.1M    8.4M        sleep  Mar04  00:01  /usr/sbin/gdm3
root         871   0.6  874.1M  113.1M        sleep  Mar04  00:53  /usr/bin/dockerd -H fd:// --containerd=/run/containerd/containerd.sock
whoopsie     875   0.1  391.4M   15.4M        sleep  Mar04  00:02  /usr/bin/whoopsie -f
kernoops     877   0.0   11.0M  444.0K        sleep  Mar04  00:01  /usr/sbin/kerneloops --test
kernoops     879   0.0   11.0M  448.0K        sleep  Mar04  00:01  /usr/sbin/kerneloops
rtkit        905   0.0  149.6M    3.1M     1  sleep  Mar04  00:00  /usr/libexec/rtkit-daemon
root        1017   0.0  246.3M    9.5M        sleep  Mar04  00:00  /usr/lib/upower/upowerd
root        1083   0.0    2.4M  576.0K        sleep  Mar04  00:00  bpfilter_umh
root        2074   0.0  163.4M    9.3M       

rmclean    11881   0.3  798.5M   52.1M        sleep  Mar04  00:06  /usr/libexec/gnome-terminal-server
rmclean    11889   0.0   11.0M    5.5M        sleep  Mar04  00:01  bash
root       14028   0.0   28.1M    9.5M        sleep  00:00  00:00  /usr/sbin/cupsd -l
root       14029   0.1  176.6M   12.2M        sleep  00:00  00:00  /usr/sbin/cups-browsed
root       24389   0.0    0.0B    0.0B         idle  04:08  00:35  kworker/u2:0-events_power_efficient
root       61607   0.0    0.0B    0.0B         idle  06:24  00:28  kworker/u2:4-events_power_efficient
root       63228   0.0    0.0B    0.0B         idle  09:31  00:13  kworker/u2:2-events_freezable_power_
root       64979   0.0    0.0B    0.0B         idle  09:57  00:11  kworker/u2:5-events_power_efficient
rmclean    65015   2.3    3.7G  457.0M        sleep  10:04  01:06  /usr/lib/firefox/firefox -new-window
rmclean    65072   0.7    2.3G  135.3M        sleep  10:04  00:08  /usr/lib/firefox/firefox -contentproc -childID 1 -isForBrowser -pr